In [1]:
get_ipython().system('pip install transformers datasets scikit-learn pandas torch')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding, pipeline


In [3]:
df = pd.read_csv("flipkart_sentiment_balanced.csv")
df.tail()

/tmp/ipython-input-1541029484.py:1: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("flipkart_sentiment_balanced.csv")


,product_name,product_price,Rate,Review,Summary,Sentiment
499738,Seagate One Touch with Password Protection for...,5599,5,classy product,it was goodbut segate software not working in ...,neutral
499739,Flipkart SmartBuy CFXB15 Electric Rice Cooker?...,1499,5,mind-blowing purchase,ok,neutral
499740,shyokand Fabric SofaFinish Color Red Preassem...,475,3,good,its k bt size to small,neutral
499741,Men LPMXT-800 Grey Clogs Sandal,519,5,NaN,recive as expected,neutral
499742,IWS Printed Double Mink Blanket for Mild Wint...,529,3,decent product,vry thin metarial,neutral


In [4]:
df.columns


Index(['product_name', 'product_price', 'Rate', 'Review', 'Summary',
       'Sentiment'],
      dtype='object')

In [5]:
df.drop(columns=['product_name', 'product_price', 'Rate', 'Review'],inplace=True)

In [6]:
df.head()

,Summary,Sentiment
0,great cooler excellent air flow and for this p...,positive
1,best budget 2 fit cooler nice cooling,positive
2,the quality is good but the power of air is de...,positive
3,very bad product its a only a fan,negative
4,ok ok product,neutral


In [7]:
# df = df[['Summary', 'Sentiment']].dropna()
# df = df[df['Sentiment'].isin(['positive', 'neutral', 'negative'])]
# df = df.sample(1000, random_state=42).reset_index(drop=True)

In [8]:
# df['airline_sentiment'].value_counts()
import warnings
warnings.filterwarnings('ignore')

In [9]:
df = (
    df[df['Sentiment'].isin(['positive', 'neutral', 'negative'])]
    .groupby('Sentiment', group_keys=False)
    .apply(lambda x: x.sample(1000, random_state=42))
    .reset_index(drop=True)
)


In [10]:
df['Sentiment'].value_counts()

,count
Sentiment,
negative,1000
neutral,1000
positive,1000


In [11]:
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['Sentiment'])

In [12]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Summary'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)


In [13]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [14]:
train_dataset = Dataset.from_dict({**train_encodings, 'label': train_labels})
val_dataset = Dataset.from_dict({**val_encodings, 'label': val_labels})


In [15]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [17]:
from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",              # where to save the model
    eval_strategy="epoch",         # evaluate at the end of each epoch
    save_strategy="epoch",               # save at the end of each epoch
    learning_rate=2e-5,                   # standard for fine-tuning
    per_device_train_batch_size=8,        # adjust based on GPU/CPU memory
    per_device_eval_batch_size=8,
    num_train_epochs=3,                   # you can change this
    weight_decay=0.01,                    # regularization
    logging_dir="./logs",                 # for TensorBoard
    logging_steps=50,
    load_best_model_at_end=True,          # load best model after training
    metric_for_best_model="accuracy",     # depends on compute_metrics
    greater_is_better=True,
    report_to="none"                     # if higher metric is better
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics

)



In [18]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics

)


In [19]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.411500,0.452155,0.816667,0.819057,0.826397,0.816667
2,0.449900,0.479001,0.846667,0.846220,0.845982,0.846667
3,0.258200,0.531507,0.850000,0.848623,0.849148,0.850000


TrainOutput(global_step=900, training_loss=0.3956942176818848, metrics={'train_runtime': 112.311, 'train_samples_per_second': 64.108, 'train_steps_per_second': 8.013, 'total_flos': 219817009699200.0, 'train_loss': 0.3956942176818848, 'epoch': 3.0})

In [21]:
eval_results = trainer.evaluate()

# Show evaluation metrics
print("📊 Evaluation Results:")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")


📊 Evaluation Results:
eval_loss: 0.5315
eval_accuracy: 0.8500
eval_f1: 0.8486
eval_precision: 0.8491
eval_recall: 0.8500
eval_runtime: 2.0812
eval_samples_per_second: 288.3000
eval_steps_per_second: 36.0380
epoch: 3.0000


In [24]:
model.save_pretrained("fp-model")
tokenizer.save_pretrained("fp-model")


('fp-model/tokenizer_config.json',
 'fp-model/special_tokens_map.json',
 'fp-model/vocab.txt',
 'fp-model/added_tokens.json',
 'fp-model/tokenizer.json')

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X = df['Summary']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

tfidf = TfidfVectorizer(max_features=3000)
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

clf = LogisticRegression()
clf.fit(X_train_vec, y_train)

y_pred = clf.predict(X_test_vec)
print(classification_report(y_test, y_pred))



              precision    recall  f1-score   support

           0       0.78      0.86      0.82       217
           1       0.74      0.68      0.71       197
           2       0.89      0.85      0.87       186

    accuracy                           0.80       600
   macro avg       0.80      0.80      0.80       600
weighted avg       0.80      0.80      0.80       600



In [29]:
 from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

# Load model and tokenizer manually
model_path = "fp-model"

model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# ✅ Fix label mapping (change these according to your training labels)
model.config.id2label = {0: "negative", 1: "neutral", 2: "positive"}
model.config.label2id = {"negative": 0, "neutral": 1, "positive": 2}

# Create pipeline with fixed label mapping
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

# Predict function
def predict_sentiment(text):
    result = classifier(text, truncation=True)[0]  # truncation to avoid length errors
    label = result['label']
    score = result['score']
    print(f"✈ Sentiment: {label} (Confidence: {score:.2f})")

# Test
test_tweet = input("Enter a tweet to analyze sentiment: ")
predict_sentiment(test_tweet)


Device set to use cuda:0


Enter a tweet to analyze sentiment: product is very bad
✈ Sentiment: negative (Confidence: 0.99)


In [30]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

# Load a high-quality pretrained sentiment model
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create pipeline
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Predict sentiment
def predict_sentiment(text):
    result = classifier(text, truncation=True)[0]
    label = result['label']
    score = result['score']
    print(f"✈ Sentiment: {label} (Confidence: {score:.2f})")

# Test
test_tweet = input("Enter a tweet to analyze sentiment: ")
predict_sentiment(test_tweet)


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0


Enter a tweet to analyze sentiment: product is very bad


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✈ Sentiment: negative (Confidence: 0.90)


In [31]:
from google.colab import files
!zip -r fp-model.zip fp-model
files.download("fp-model.zip")


  adding: fp-model/ (stored 0%)
  adding: fp-model/config.json (deflated 48%)
  adding: fp-model/tokenizer.json (deflated 71%)
  adding: fp-model/model.safetensors (deflated 8%)
  adding: fp-model/vocab.txt (deflated 53%)
  adding: fp-model/tokenizer_config.json (deflated 75%)
  adding: fp-model/special_tokens_map.json (deflated 42%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>